In [278]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
from sklearn.metrics import auc, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import GridSearchCV


K=2
lamU=0.1
lamV=0.1

lr = 0.001 #step size i.e. learning rate
MAX_ITER = 10 #200

path = "../dataset/data.npz"


In [248]:
npzfile.files

['item_id', 'user_id', 'rating']

In [249]:
#data combination and split. implement based on numpy API
#
dataAll = np.asarray([npzfile['user_id'].reshape(-1), npzfile['item_id'].reshape(-1), npzfile['rating'].reshape(-1)] )
dataAll=dataAll.T
[USER_ID_MAX, ITEM_ID_MAX, _] = np.max(dataAll, axis=0) #TODO: n unique
trainData, testData = train_test_split(dataAll, test_size=0.2, random_state=0)


In [ ]:
R_train = dataToMatrix(trainData)
indicator = (R_train > 0).astype(int)
np.random.seed(seed=1) # for reproducible
#initialize params
U= np.random.random_sample((K, USER_ID_MAX))
V= np.random.random_sample((K, ITEM_ID_MAX))

R_test = dataToMatrix(testData)
indicatoR_test = (R_test>0).astype(int)


R_pred = U.T @ V # matmul, TODO: logistic function to overcome rating out range
diff = indicator * (R_train - R_pred)

rmseTrain=[]
rmseTest=[]


for epoch in range(MAX_ITER):
     #TODO: Linear with # of rating. take advantage of the sparsity of the matrix.
    dEdU = -V @ diff.T  + lamU * U
    dEdV = -U @ diff  + lamV * V
    # sync update 
    Ut = U - lr * dEdU #TODO:  - or +
    Vt = V - lr * dEdV
    U = Ut
    V = Vt
    
    R_pred = U.T @ V #matmul, TODO: logistic function to overcome rating out range
    diff = indicator * (R_train - R_pred)
    
    # evaluate RMSE loss
    rmseTrain.append(np.sqrt( (diff**2).sum()/indicator.sum()))
    diffTest = indicatoR_test * (R_test - R_pred)
    rmseTest.append(np.sqrt( (diffTest**2).sum()/indicatoR_test.sum()))
    print("Epoch {}: Train RMSE: {:.4f}\t Test RMSE: {:.4f}".format(epoch, rmseTrain[epoch], rmseTest[epoch]))



In [384]:
#training and infer without cross validation
# note that idx starts from 0. user_id=1 ->  row 0 in the matrix
def dataToMatrix(data, dim=[USER_ID_MAX, ITEM_ID_MAX]):
    matrix = np.zeros(dim)
    for ins in data:
        matrix[ins[0]-1,ins[1]-1] = ins[2]
    return matrix

#rawData -input data with format [[userId, itemId, rating]xN]
def PMF_test(rawData, U, V, useSparse=True):
    #TODO: use sparsity.
    if(not useSparse):
        R_test = dataToMatrix(rawData)
        indicatoR_test = (R_test>0).astype(int)
        R_pred = U.T @ V 
        diff = indicatoR_test * (R_test - R_pred)
        rmse = np.sqrt( (diff**2).sum()/indicatoR_test.sum())
    else:
        mseSum = 0
        for (uID, iID, rij) in rawData:
            i = uID - 1
            j = iID - 1
            mseSum += (rij - U.T[i,:]@V[:,j])**2
        rmse = np.sqrt(mseSum/len(rawData))
    return rmse


#rawData -input data with format [[userId, itemId, rating]xN]
#K - the number of latent features
def PMF_train(rawData, maxIter=200, K=2,  lamU=0.1, lamV=0.1, useSparse=False ,verbose=0):
    
    #initialize params. 
    np.random.seed(seed=1)
    U= np.random.random_sample((K, USER_ID_MAX)) 
    V= np.random.random_sample((K, ITEM_ID_MAX))
    n_iter = 0
    
    if(not useSparse):
        # Matrix Operations
        R_train = dataToMatrix(rawData)
        indicator = (R_train > 0).astype(int)

        R_pred = U.T @ V # matmul, TODO: logistic function to overcome rating out range
        diff = indicator * (R_train - R_pred)

        rmseTrain=[]
        for epoch in range(maxIter):
             #TODO: Linear with # of rating. take advantage of the sparsity of the matrix.
            dEdU = -V @ diff.T  + lamU * U
            dEdV = -U @ diff  + lamV * V
            # sync update 
            Ut = U - lr * dEdU #TODO:  - or +
            Vt = V - lr * dEdV
            U = Ut
            V = Vt

            R_pred = U.T @ V #matmul, TODO: logistic function to overcome rating out range
            diff = indicator * (R_train - R_pred)

            n_iter +=1

            rmseTrain.append(np.sqrt( (diff**2).sum()/indicator.sum()))
            # evaluate RMSE loss
            if(verbose):
                print("Epoch {}: Train RMSE: {:.4f}".format(epoch, rmseTrain[epoch]))
    else:
        #Running Time is Linear to observed rating
        rmseTrain = np.zeros(maxIter)
        for epoch in range(maxIter):
            Ut = U
            Vt = V
            for (uID,iID,rij) in trainData:
                i = uID - 1
                j = iID - 1
                #partially update the latent features of user i and item j
                diff_ij = rij - (U.T)[i,:] @ V[:,j]
                Ut[:,i] += - lr * (-V[:,j] * diff_ij) 
                Vt[:,j] += - lr * (-U[:,i] * diff_ij)
                rmseTrain[epoch] += diff_ij**2
            #update U and V
            Ut = Ut - lr * ( lamU * U)
            Vt = Vt - lr * (lamV * V)
            U = Ut
            V = Vt
            
            rmseTrain[epoch] = np.sqrt(rmseTrain[epoch]/len(trainData))
            n_iter+=1
            if(verbose):
                print("Epoch {}: Train RMSE: {:.4f}".format(epoch, rmseTrain[epoch]))
        
    return U,V, n_iter, rmseTrain[n_iter-1]
    
U,V,n_iter, rmse = PMF_train(testData,K=2, maxIter=100, useSparse=False, verbose=1)
PMF_test(trainData, U, V)
    

Epoch 0: Train RMSE: 3.1344
Epoch 1: Train RMSE: 3.0064
Epoch 2: Train RMSE: 2.8634
Epoch 3: Train RMSE: 2.7085
Epoch 4: Train RMSE: 2.5466
Epoch 5: Train RMSE: 2.3841
Epoch 6: Train RMSE: 2.2280
Epoch 7: Train RMSE: 2.0842
Epoch 8: Train RMSE: 1.9563
Epoch 9: Train RMSE: 1.8457
Epoch 10: Train RMSE: 1.7515
Epoch 11: Train RMSE: 1.6718
Epoch 12: Train RMSE: 1.6039
Epoch 13: Train RMSE: 1.5456
Epoch 14: Train RMSE: 1.4950
Epoch 15: Train RMSE: 1.4507
Epoch 16: Train RMSE: 1.4114
Epoch 17: Train RMSE: 1.3763
Epoch 18: Train RMSE: 1.3447
Epoch 19: Train RMSE: 1.3162
Epoch 20: Train RMSE: 1.2903
Epoch 21: Train RMSE: 1.2667
Epoch 22: Train RMSE: 1.2451
Epoch 23: Train RMSE: 1.2252
Epoch 24: Train RMSE: 1.2069
Epoch 25: Train RMSE: 1.1901
Epoch 26: Train RMSE: 1.1744
Epoch 27: Train RMSE: 1.1599
Epoch 28: Train RMSE: 1.1464
Epoch 29: Train RMSE: 1.1339
Epoch 30: Train RMSE: 1.1221
Epoch 31: Train RMSE: 1.1112
Epoch 32: Train RMSE: 1.1009
Epoch 33: Train RMSE: 1.0913
Epoch 34: Train RMSE: 1.

1.0470970883356563

In [378]:
class PMF(BaseEstimator,TransformerMixin): #TODO: Transformer needed?
    def __init__(self, maxIter=200, K=2, lamU=0.1, lamV=0.1 ):
        self.maxIter = maxIter
        self.K = K
        self.lamU = lamU
        self.lamV = lamV
        
        self.U = np.random.random_sample((K, USER_ID_MAX))
        self.V = np.random.random_sample((K, ITEM_ID_MAX))
        
    
    # interface for estimator
    def fit(self, X, y=None, **params):
        U, V, n_iter_, train_rmse_ = PMF_train(X, maxIter=self.maxIter, K=self.K, lamU=self.lamU, lamV=self.lamV)
        
        #parameters with trailing _ is used to check if the estimator has been fitted
        #TODO: add validation rmse
        self.rmse_=  train_rmse_
        self.n_iter_ = n_iter_
        self.U = U
        self.V = V
        
        return self
    
    # interface for Grid Search
    def score(self, X, y=None):
        rmse = PMF_test(X, self.U, self.V)
        #print("Scoring: K={}, lamda U={}, lamda V={}, rmse={}".format(self.K,self.lamU, self.lamV, rmse) )
        #since build-in gridsearch pick params by "the bigger the better"
        return -rmse
        
def GridSearchTunning(trainData, maxIter=100, verbose=0):
    #Tune regularization hyper-params
    regu_params = {"lamU":[0.1,1,10,100],"lamV":[0.1,1,10,100]}
    pmfEst = PMF(K=2, maxIter=maxIter)
    #splitting data into train/validation set by 5-fold 
    gs=GridSearchCV(pmfEst, regu_params, cv=5, refit=True, verbose=verbose)
    gs.fit(trainData)

    bestLamU = gs.best_params_['lamU']
    bestLamV = gs.best_params_['lamV']
    #Mean cross-validated score of the best_estimator
    bestScoreL = -gs.best_score_ 
    print("Finish tunning lamda U and lamda V \r\n==> Best lamU {}, best lamV {}, RMSE={}".format(bestLamU, bestLamV,bestScoreL ))
    
    #Tune # latent features
    factors_params = {"K":[1,2,3,4,5]}
    pmfEst2 = PMF(lamU=bestLamU,lamV=bestLamV, maxIter=maxIter)
    gs2 =GridSearchCV(pmfEst, factors_params, cv=5, refit=True, verbose=verbose)
    gs2.fit(trainData)

    bestK = gs2.best_params_['K']
    bestScoreK = -gs2.best_score_
    print("Finish tunning factors K \r\n==> Best K={}, RMSE={}".format(bestK,bestScoreK ))
    
    if(verbose==2):
        print("CV details:{}\r\n{}".format(gs.cv_results_, gs2.cv_results_) )
    
    return bestLamU, bestLamV, bestK

#experiment on a given train data and test data
def Experiment(trainData, testData, verbose=0):
    #Grid Search Tunning on training set
    bestLamU, bestLamV, bestK = GridSearchTunning(trainData, maxIter=100, verbose=verbose)
    #train with the full training set using the tuned best hyper-params
    U,V,_,rmse_train = PMF_train(trainData, K=bestK, lamU=bestLamU, lamV=bestLamV, verbose=1 )
    #evaluate on test set
    rmse_test = PMF_test(testData, U,V)
    
    return rmse_test
    
def main():
    
    trainData, testData = train_test_split(dataAll, test_size=0.2, random_state=0)
    # Dense training data
    rmse1 = Experiment(trainData, testData)
    print("RMSE of test set on dense data: {:.4f}".format(rmse1))
    # Sparse training data
    rmse2 = Experiment(testData, trainData)
    print("RMSE of test set on sparse data: {:.4f}".format(rmse2))
    
    
    

In [355]:
#Tune hyper-params with Grid Search

MAX_ITER=100
#Tune regularization hyper-params
regu_params = {"lamU":[0.1,1,10,100],"lamV":[0.1,1,10,100]}
pmfEst = PMF(K=2, maxIter=MAX_ITER)
gs=GridSearchCV(pmfEst, regu_params, cv=5, refit=True, verbose=1)
gs.fit(trainData)

bestLamU = gs.best_params_['lamU']
bestLamV = gs.best_params_['lamV']
#Mean cross-validated score of the best_estimator
bestScoreL = -gs.best_score_
print("best lamU {}, best lamV {}, RMSE={}".format(bestLamU, bestLamV,bestScoreL ))
print(gs.best_params_)
print(gs.cv_results_)

#Tune # latent features
factors_params = {"K":[1,2,3,4,5]}
pmfEst2 = PMF(lamU=bestLamU,lamV=bestLamV, maxIter=MAX_ITER)
gs2 =GridSearchCV(pmfEst, factors_params, cv=5, refit=True, verbose=1)
gs2.fit(trainData)

bestK = gs2.best_params_['K']
bestScoreK = -gs2.best_score_
print("best K={}, RMSE={}".format(bestK,bestScoreK ))


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  3.5min finished


best lamU 0.1, best lamV 0.1, RMSE=-0.9571893631715415
{'lamU': 0.1, 'lamV': 0.1}
{'split0_test_score': array([-0.95849969, -0.95908187, -0.97517916, -1.91647734, -0.95944245,
       -0.96026188, -0.97825638, -1.24697082, -0.97571203, -0.97841153,
       -1.01374915, -1.37724476, -1.38552123, -2.02520265, -1.31341168,
       -1.83076559]), 'split1_test_score': array([-0.95061414, -0.95149337, -0.97050946, -3.01952477, -0.95152726,
       -0.95265752, -0.97364229, -1.33661307, -0.96825875, -0.97134193,
       -1.0099879 , -1.88824764, -2.19279159, -1.39581863, -1.36210226,
       -1.83782816]), 'split2_test_score': array([-0.95151775, -0.9519353 , -0.96721808, -1.53972565, -0.95224076,
       -0.95290902, -0.97017395, -1.26512344, -0.96746673, -0.97009028,
       -1.00517737, -1.43262981, -3.33790947, -1.2773578 , -1.27041385,
       -1.82537645]), 'split3_test_score': array([-0.95903581, -0.95970445, -0.97635907, -2.44732053, -0.95991132,
       -0.96082726, -0.97947702, -1.41412145, -

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  1.1min finished


best K=2, RMSE=-0.9571893631715415
{'lamU': 0.1, 'lamV': 0.1}
{'split0_test_score': array([-0.95849969, -0.95908187, -0.97517916, -1.91647734, -0.95944245,
       -0.96026188, -0.97825638, -1.24697082, -0.97571203, -0.97841153,
       -1.01374915, -1.37724476, -1.38552123, -2.02520265, -1.31341168,
       -1.83076559]), 'split1_test_score': array([-0.95061414, -0.95149337, -0.97050946, -3.01952477, -0.95152726,
       -0.95265752, -0.97364229, -1.33661307, -0.96825875, -0.97134193,
       -1.0099879 , -1.88824764, -2.19279159, -1.39581863, -1.36210226,
       -1.83782816]), 'split2_test_score': array([-0.95151775, -0.9519353 , -0.96721808, -1.53972565, -0.95224076,
       -0.95290902, -0.97017395, -1.26512344, -0.96746673, -0.97009028,
       -1.00517737, -1.43262981, -3.33790947, -1.2773578 , -1.27041385,
       -1.82537645]), 'split3_test_score': array([-0.95903581, -0.95970445, -0.97635907, -2.44732053, -0.95991132,
       -0.96082726, -0.97947702, -1.41412145, -0.97635065, -0.97921

In [377]:
# debug session
Experiment(testData,trainData,verbose=1)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.2min finished


Finish tunning lamda U and lamda V 
==> Best lamU 0.1, best lamV 0.1, RMSE=1.0722257920860467
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   39.4s finished


Finish tunning factors K 
==> Best K=5, RMSE=1.0379674940307944
Epoch 0: Train RMSE: 2.3937
Epoch 1: Train RMSE: 2.1956
Epoch 2: Train RMSE: 2.0086
Epoch 3: Train RMSE: 1.8406
Epoch 4: Train RMSE: 1.6968
Epoch 5: Train RMSE: 1.5783
Epoch 6: Train RMSE: 1.4831
Epoch 7: Train RMSE: 1.4074
Epoch 8: Train RMSE: 1.3469
Epoch 9: Train RMSE: 1.2980


1.3697991719776725

In [251]:
#linear running time in observed ratings. using the sparsity
np.random.seed(seed=1)
U= np.random.random_sample((K, USER_ID_MAX))
V= np.random.random_sample((K, ITEM_ID_MAX))
MAX_ITER2=2
for epoch in range(MAX_ITER2):
    # linear algo. Batch Gradient Descent
    Ut = U
    Vt = V
    for (userID,itemID,rij) in trainData:
        i = userID - 1
        j = itemID - 1
        rij_pred = np.dot((U.T)[i,:],V[:,j])
        diff_ij = rij - rij_pred
        #partially update the latent features of user i and item j
        Ut[:,i] += - lr * (-V[:,j] * diff_ij) 
        Vt[:,j] += - lr * (-U[:,i] * diff_ij)
        
        #rmseTrain[epoch] += diff_ij**2
    #update U and V
    Ut = Ut - lr * ( lamU * U)
    Vt = Vt - lr * (lamV * V)
    U = Ut
    V = Vt
    
    #evaluate
    #rmseTrain[epoch] = np.sqrt(rmseTrain[epoch]/len(trainData))
    R_pred = U.T @ V
    diffTrain = indicator * (R_train - R_pred)
    rmseTrain[epoch] = np.sqrt( (diffTrain**2).sum()/indicator.sum())
    diffTest = indicatoR_test * (R_test - R_pred)
    rmseTest[epoch] = np.sqrt( (diffTest**2).sum()/indicatoR_test.sum())
    print("Epoch {}: Train RMSE: {:.4f}\t Test RMSE: {:.4f}".format(epoch, rmseTrain[epoch], rmseTest[epoch]))
    
###===> The running time is longer than that of using the complete matrix    
        

Epoch 0: Train RMSE: 2.7059	 Test RMSE: 2.7170
Epoch 1: Train RMSE: 2.1115	 Test RMSE: 2.1261
Epoch 2: Train RMSE: 1.7169	 Test RMSE: 1.7300
Epoch 3: Train RMSE: 1.4985	 Test RMSE: 1.5105
Epoch 4: Train RMSE: 1.3645	 Test RMSE: 1.3766
Epoch 5: Train RMSE: 1.2731	 Test RMSE: 1.2862
Epoch 6: Train RMSE: 1.2072	 Test RMSE: 1.2214
Epoch 7: Train RMSE: 1.1577	 Test RMSE: 1.1732
Epoch 8: Train RMSE: 1.1197	 Test RMSE: 1.1364
Epoch 9: Train RMSE: 1.0897	 Test RMSE: 1.1077


In [253]:
# for python test
np.random.seed(seed=1)
a = np.random.random_sample((2,2))
b = (a > 0).astype(int)
b[0,0]=0
print(RMatrix.shape)
print(V.shape)
print(U.shape)
dt=np.asarray([1,1])
indicator.sum()


(943, 1682)
(2, 1682)
(2, 943)


80000